In [16]:
import ebest_api2 as eba
import auth
import redis
from datetime import datetime
import threading
import logging
from time import gmtime, strftime
import pandas as pd
import time
import requests
import urllib3
from urllib3.exceptions import InsecureRequestWarning
urllib3.disable_warnings(InsecureRequestWarning)
import pymysql

today_date = datetime.now().strftime('%Y-%m-%d')

filename = './푸드_'+ today_date+'.log'

logging.basicConfig(filename=filename,filemode='a+',level=logging.INFO)

ACCESS_TOKEN=eba.make_token()
acc = eba.acc_info(ACCESS_TOKEN)

### 매수(obj_CpTrade_CpTd0311)

In [18]:
balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')
balance

400246269

In [20]:
def log_it(now_time, info):
    o = str(now_time) +' '+ str(info)
    logging.info(o)

In [21]:
## redis 받는 행동
def get_redis(get_channel_name):

    r = redis.Redis(host='34.64.37.236', port=6379, db=0)

    # subscribe
    s = r.pubsub()
    s.subscribe(get_channel_name)

    i = 0
    while True :
        res = s.get_message()
        for message in s.listen():
            data = message.get("data")
            if isinstance(data, bytes):
                i=i+1
                break

        if i==1:
            break
    data = str(data,'utf-8')
    return data

In [23]:
def insert_pv(date, pv):
    conn = pymysql.connect(host='34.64.197.181', user='SUJIN', password='SUJIN',
    db='shin_db', charset='utf8')
    curs = conn.cursor()
    sql = '''INSERT IGNORE INTO ShinFoodPred VALUES (%s, %s)'''
    
#     for i in range(shin.shape[0]):
    val = (date, pv)
    curs.execute(sql, val)
    conn.commit()
    conn.close()

In [24]:
## redis 보내는 행동
def send_redis(send_channel_name, info): # balance_channel으로 할거임

    r = redis.Redis(host='34.64.37.236', port=6379, db=0)

    # print(info)

    r.publish(send_channel_name, info)

In [25]:
## redis 보내는 행동
def chan_send_redis(send_channel_name, info):

    r = redis.Redis(host='34.64.63.106', port=6379, db=0)

    r.publish(send_channel_name, info)

In [27]:
# eba.order(ACCESS_TOKEN,'A031440','buy',1)

In [28]:
def autobuy(stock):
    ACCESS_TOKEN=eba.make_token()
    all=[]
    now_time5 = datetime.now().strftime('%H%M')
    while int(now_time5) <=1520:
        now_time5 = datetime.now().strftime('%H%M')


        k=eba.trade_order(ACCESS_TOKEN)
        t=k.get('CSPAQ13700OutBlock3')

        print("=======================================")
        print(stock)
        now_min1 = datetime.now().strftime('%M')
        now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')    
        acc = eba.acc_info(ACCESS_TOKEN)
        
        if 'CSPAQ12300OutBlock2' in acc.keys():
            print("[API] 잔고를 가져옵니다")
            balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')
        
        print("[시스템] 현재시각은 {}입니다".format(now_time))
        print('[시스템] 남은 잔고는 {}입니다'.format(balance))
        send_redis(stock+'balance_channel', balance)
        log_it(now_time, balance)
        print("[Redis] action을 기다리는 중입니다")
        action = get_redis(stock+'action_channel')
        print("[Redis] unit을 기다리는 중입니다")
        unit = get_redis(stock+'trading_channel')
        unit = int(unit)
        log_it(now_time, action)
        log_it(now_time, unit)

        print("[API] 체결단가를 가져오는중..")
        while True:
            ACCESS_TOKEN=eba.make_token()
            trade = eba.trade_order(ACCESS_TOKEN)
            time.sleep(1)
            if 'CSPAQ13700OutBlock3' in trade.keys():  
                t=trade.get('CSPAQ13700OutBlock3')
                for i,li in enumerate(t):
                    if li.get('IsuNm')=='신세계푸드':
                        all.append(li)

                print("[API] 체결 단가를 가져옵니다")
                real_close1=str(int(float(all[0].get('ExecPrc'))))
                break

        che_lsit = eba.chelist(ACCESS_TOKEN,'031440')
        real_close1 = str(che_lsit['t0425OutBlock1'][-1]['cheprice'])
        print('수정된1 real_close:'+real_close1)


        while action=='0':
            real_close = real_close1
            now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('\n')
            print("=======================================")
            print('현재시각 {} 관망합니다'.format(now_time))
            insert_pv(now_time,'관망')
            time.sleep(15)
            send_redis(stock+'balance_channel', balance)
            print('[Redis] 보내는 잔고는 {}입니다'.format(balance))
            log_it(now_time, balance)

            time.sleep(5)
            send_redis(stock+'real_close', real_close)
            print('[Redis] 보내는 종가는 {}입니다'.format(real_close))
            log_it(now_time, real_close)

            print("[Redis] action을 기다리는 중입니다")
            action = get_redis(stock+'action_channel')
            print("[Redis] unit을 기다리는 중입니다")
            unit = get_redis(stock+'trading_channel')
            unit = int(unit)
            log_it(now_time, '관망')
            log_it(now_time, unit)

            if action!='0':
                break

##################### 매매하는 부분
        if action =='2':
            ACCESS_TOKEN=eba.make_token()
            now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('\n')
            print("=======================================")
            print('[시스템] 현재시각 {}, {}개를 매수합니다'.format(now_time, unit))
            log_it(now_time, '매수')
            print(eba.order(ACCESS_TOKEN,'A'+stock,'buy',unit))
            insert_pv(now_time,'매수')

        if action =='1':
            ACCESS_TOKEN=eba.make_token()
            now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('\n')
            print("=======================================")
            print('[시스템] 현재시각 {}, {}개를 매도합니다'.format(now_time, unit))
            log_it(now_time, '매도')
            print(eba.order(ACCESS_TOKEN,'A'+stock,'sell',unit))
            insert_pv(now_time,'매도')
####################     

        if 'CSPAQ12300OutBlock2' in acc.keys():
            print("[API] 잔고를 가져옵니다")
            balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')

        print("[API] 체결단가를 가져오는중..")
        time.sleep(5)
        while True:
            time.sleep(1)
            ACCESS_TOKEN=eba.make_token()
            trade = eba.trade_order(ACCESS_TOKEN)

            if 'CSPAQ13700OutBlock3' in trade.keys():  
                t=trade.get('CSPAQ13700OutBlock3')
                for i,li in enumerate(t):
                    if li.get('IsuNm')=='신세계푸드':
                        all.append(li)

                print("[API] 체결 단가를 가져옵니다")
                real_close2=str(int(float(all[0].get('ExecPrc'))))
                break
            
        che_lsit = eba.chelist(ACCESS_TOKEN,'031440')
        real_close2 = str(che_lsit['t0425OutBlock1'][-1]['cheprice'])
        print('수정된2 real_close:'+real_close2)


        if real_close2=='0':
            print('real_close1로 들어옵니다')
            real_close = real_close1
        else:
            print('real_close2로 들어옵니다')
            real_close = real_close2

        print('[Redis] 남은 잔고는 {}입니다 보냅니다'.format(balance))
        time.sleep(1)
        send_redis(stock+'balance_channel', balance)
        now_time = datetime.now().strftime('%H:%M:%S')
        log_it(now_time, balance)
        print('[Redis] 체결단가는 {}입니다 학습하게 보냅니다'.format(real_close))
        time.sleep(5)
        send_redis(stock+'real_close', real_close)
        now_time = datetime.now().strftime('%H:%M:%S')
        log_it(now_time, real_close)

In [29]:
autobuy('031440')

031440
[API] 잔고를 가져옵니다
[시스템] 현재시각은 2023-08-08 09:26:05입니다
[시스템] 남은 잔고는 400246269입니다
[Redis] action을 기다리는 중입니다
[Redis] unit을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된1 real_close:0


[시스템] 현재시각 2023-08-08 09:26:24, 1개를 매도합니다
모의투자 매도주문이 완료 되었습니다.
[API] 잔고를 가져옵니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된2 real_close:0
real_close1로 들어옵니다
[Redis] 남은 잔고는 400246269입니다 보냅니다
[Redis] 체결단가는 0입니다 학습하게 보냅니다
031440
[API] 잔고를 가져옵니다
[시스템] 현재시각은 2023-08-08 09:26:38입니다
[시스템] 남은 잔고는 400246269입니다
[Redis] action을 기다리는 중입니다
[Redis] unit을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된1 real_close:41450


[시스템] 현재시각 2023-08-08 09:27:41, 1개를 매도합니다
모의투자 매도주문이 완료 되었습니다.
[API] 잔고를 가져옵니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된2 real_close:0
real_close1로 들어옵니다
[Redis] 남은 잔고는 400246269입니다 보냅니다
[Redis] 체결단가는 41450입니다 학습하게 보냅니다
031440
[API] 잔고를 가져옵니다
[시스템] 현재시각은 2023-08-08 09:27:54입니다
[시스템] 남은 잔고는 400292603입니다
[Redis] action을 기다리는 중입니다
[Redis] unit을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다